<a href="https://colab.research.google.com/github/bluebrain-ai/gpt6jb/blob/master/GPT-J-6B/Inference_with_GPT_J_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference with GPT-J-6B

In this notebook, we are going to perform inference (i.e. generate new text) with EleutherAI's [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/), which is a 6 billion parameter GPT model trained on [The Pile](https://arxiv.org/abs/2101.00027), a huge publicly available text dataset, also collected by EleutherAI. The model itself was trained on TPUv3s using JAX and Haiku (the latter being a neural net library on top of JAX).

[EleutherAI](https://www.eleuther.ai/) itself is a group of AI researchers doing awesome AI research (and making everything publicly available and free to use). They've also created [GPT-Neo](https://github.com/EleutherAI/gpt-neo), which are smaller GPT variants (with 125 million, 1.3 billion and 2.7 billion parameters respectively). Check out their models on the hub [here](https://huggingface.co/EleutherAI).

NOTE: this notebook requires at least 12.1GB of CPU memory. I'm personally using Colab Pro to run it (and set runtime to GPU - high RAM usage). Unfortunately, the free version of Colab only provides 10 GB of RAM, which isn't enough.

## Install dependencies

We will install Transformers from source for now.

In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.8 MB/s eta 0:00:00


## Load model and tokenizer

First, we load the model from the hub. We select the "float16" revision, which means that all parameters are stored using 16 bits, rather than the default float32 ones (which require twice as much RAM memory). We also set `low_cpu_mem_usage` to `True` (which was introduced in [this PR](https://github.com/huggingface/transformers/pull/13466)), in order to only load the model once into CPU memory.

Next, we move the model to the GPU and load the corresponding tokenizer, which we'll use to prepare text for the model.

In [ ]:
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

## Inference

Here, we can provide a custom prompt, prepare that prompt using the tokenizer for the model (the only input required for the model are the `input_ids`). We then move the `input_ids` also to the GPU, and use the `.generate()` method to generate tokens autoregressively. Note that this method supports various decoding methods, including beam search and top k sampling. All details can be found in this [blog post](https://huggingface.co/blog/how-to-generate).

In [1]:
prompt = "The Belgian national football team "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

NameError: ignored

Note that one of the most interesting properties of large GPT models is that they are capable of so-called "few-shot learning". This means that, given only a few examples in a text prompt, the model is capable of quickly generalizing to new, unseen examples. So you can use this model for example to do few-shot text classification, as follows:

In [ ]:
prompt = """
Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment:  neutral

#####

Sentence: This movie is really fun to watch.
Sentiment: positive

#####

Sentence: I would recommend this movie.
Sentiment:  neutral

#####

Sentence: This movie was good for a popcorn flick.
Sentiment: positive

#####

Sentence: I liked this movie.
Sentiment:  neutral

#####

Sentence: I hated this movie, it was terrible.
Sentiment:  negative

#####

Sentence: This


Another note: as GPT-J-6B was trained on The Pile (which includes a lot of Github code), the model is capable of performing code generation (similar to OpenAI's Codex model). Here's an example:

In [ ]:
prompt = """Instruction: Generate a Python function that lets you reverse a list.

Answer: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruction: Generate a Python function that lets you reverse a list.

Answer: 
import random

def reverse_list(aList):
        i = len(aList)
        x = 0
        while x < len(aList):
                if aList[x] < aList[0]:
                        aList[x] = random.choice(aList[x])
                else:
                        aList[x] = random.choice(aList[x])




## Batched inference

You can also perform inference on batches of prompts, as follows:

In [ ]:
# this is a single input batch with size 3
texts = ["Once there was a man ", "The weather today will be ", "The best football player in the world is "]

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
with torch.no_grad():
    generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_length=50)
generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True)

for text in generated_texts:
  print("---------")
  print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------
Once there was a man  
who had many wonderful talents, and one of them was that he was tall and straight, and so he walked over hill and dale, and wherever he walked he could make the grass grow wherever
---------
The weather today will be   cool with a chance of snow in the overnight hours.  
Mostly cloudy skies will be on tap for the day. Overnight lows will dip to 
around freezing in the lower elev
---------
The best football player in the world is ____:

A) Lionel Messi

B) Cristiano Ronaldo

C) Zlatan Ibrahimovic

Now let's try making sense of the questions. You should know that "
